In [313]:
from langchain.indexes import GraphIndexCreator
# from langchain_openai import OpenAI
from langchain_community.llms import Ollama

import networkx as nx


from dotenv import load_dotenv

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from operator import itemgetter

from langchain_community.graphs.networkx_graph import NetworkxEntityGraph
from langchain_community.graphs.networkx_graph import KnowledgeTriple

from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [314]:
load_dotenv()

True

In [315]:
model = Ollama(model="llama2")

In [316]:
output_parser = StrOutputParser()

In [317]:
# read graph as entity graph
G = NetworkxEntityGraph()

In [318]:
G.add_triple(KnowledgeTriple('Charging Station', 'connects to', 'N1'))
G.add_triple(KnowledgeTriple('N1', 'connects to', 'Mensa'))
G.add_triple(KnowledgeTriple('N1', 'connects to', 'N2'))
G.add_triple(KnowledgeTriple('Mensa', 'connects to', 'N3'))
G.add_triple(KnowledgeTriple('N2', 'connects to', 'N3'))
G.add_triple(KnowledgeTriple('N2', 'connects to', 'Blood Storage'))
G.add_triple(KnowledgeTriple('N2', 'connects to', 'N4'))
G.add_triple(KnowledgeTriple('N3', 'connects to', 'N4'))
G.add_triple(KnowledgeTriple('N3', 'connects to', 'Waste Disposal'))

In [319]:
[(triple[0], triple[1]) for triple in G.get_triples()]


[('Charging Station', 'N1'),
 ('N1', 'Mensa'),
 ('N1', 'N2'),
 ('Mensa', 'N3'),
 ('N2', 'N3'),
 ('N2', 'Blood Storage'),
 ('N2', 'N4'),
 ('N3', 'N4'),
 ('N3', 'Waste Disposal')]

In [320]:
template = f"Answer the question based only on the following context: We have a directed graph. It contains these edges: {[(triple[0], triple[1]) for triple in G.get_triples()]} \n Question: {{question}}"

In [321]:
#prompt = ChatPromptTemplate.from_template(template)

In [322]:
# create chain
#chain = ({"question": RunnablePassthrough()}
#        | prompt
#        | model
#        | StrOutputParser()
#)

In [323]:
#chain.invoke("What are the options to get from Charging Station to Waste Disposal? Can you list the routes?")

In [324]:
#chain.invoke("Assume that node N2 is not available any more. Could you update the list of edges, and provide only that list in this format \{(\'Edge1\':\'ConnectingEdge1\'),(\'Edge2\':\'ConnectingEdge2\')...\}?")
#chain.invoke("Assume that node N2 is not available any more and so it has no more connections make sure N2 doesn't appear anymore. Could you update the list of edges, use this format: {(\'Edge1\',\'ConnectingEdge1\'),(\'Edge2\',\'ConnectingEdge2\')...\}?")


In [327]:
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMCheckerChain

template = f"Answer the question based only on the following context: We have a directed graph. It contains only these edges: {[(triple[0], triple[1]) for triple in G.get_triples()]} and you and have to act like inaccesible nodes and all edges containing that node don't exist, you can't add new edges or update existing edges \n Question: {{question}}"
template_new=f"You are a master graph modeler, you have to remove edges from an edge list that contain inaccesible nodes and it is crucial make sure that the other valid edges remain in the list. We have a directed graph. It contains exclusively these edges: {[(triple[0], triple[1]) for triple in G.get_triples()]} \n Question: {{question}}"
#prompt_template = PromptTemplate(input_variables=["question"], template=template)
prompt_template = PromptTemplate(input_variables=["question"], template=template_new)

new_graph=LLMChain(prompt=prompt_template,llm=model)
#new_graph2=LLMCheckerChain(prompt=prompt_template,llm=model)

route_template="{edges}, What are the routes to get from Charging Station to Waste Disposal given the new list of edges? List the routes step by step, as comma separated values"
prompt_route=PromptTemplate(input_variables=["edges"],template=route_template)
new_route=LLMChain(prompt=prompt_route,llm=model)
#new_route2=LLMCheckerChain(prompt=prompt_route,llm=model)


print([(triple[0], triple[1]) for triple in G.get_triples()])

#ssc= SimpleSequentialChain(chains=[new_graph,new_route],verbose=True)
#ssc= SimpleSequentialChain(chains=[new_graph,new_route],verbose=True)
Node='N2'
question=" Node "+"'"+ str(Node) +"'"+" is not accesible anymore. Could you provide me only with the edges that have to be removed, as a comma separated value list of tuples like (Node1,Node2)?"
print(question)
#new_graph.run("You are a master graph modeler, you have to remove edges from an edge list that contain an inaccesible node. We have a directed graph. It contains exclusively these edges: "+ str([(triple[0], triple[1]) for triple in G.get_triples()]) + " Node 'N2' is not accesible anymore. Could you, without telling me the remaining edges, provide me only with the edges that have to be removed, as a comma separated value list of tuples like (Node1,Node2)?")
#####new_graph.run(" Node 'N2' is not accesible anymore. Could you provide me only with the edges that have to be removed, as a comma separated value list of tuples like (Node1,Node2)?")
answer=new_graph.run(question)
#new_graph.run("We have a directed graph. It contains exclusively these edges: " + str([(triple[0], triple[1]) for triple in G.get_triples()]) +"Given that 'N2' is not accesible anymore. Could you provide me with the edges that should be removed, as comma separated values like this [(Node1,Node2),...]?")

#ssc.run("Given that 'N2' is not accesible anymore. Could you provide me with the edges that should be removed, as comma separated values?")
import re
print(answer)
pattern = r"\('remov.*?:.*?\n*([^\n']+)'\)"
pattern2 = r"remov.*?:.*?([^\n]+)"
pattern3 = r"\(([^\)]*)\)"


# Find all matches in the text
removed_edges = re.findall(pattern2, answer,re.DOTALL)

print("List of removed edges:", removed_edges)
print(removed_edges)
print(len(removed_edges))
for item in removed_edges:
    print(item)
edges_list = re.split(r",(?![^()]*\))", removed_edges[0])
print(edges_list)
print(len(edges_list))
#print(removed_edges.split(','))

#checker_chain=LLMCheckerChain.from_llm(llm=model,verbose=True)
#checker_chain.run("You are a master graph modeler, you have to remove edges from an edge list that contain inaccesible nodes. We have a directed graph. It contains exclusively these edges: " +str([(triple[0], triple[1]) for triple in G.get_triples()]) + "Given that 'N2' is not accesible anymore. Please provide the edges that should be removed from the edge list as comma separated value like this [(X,Y),...]? Use the format: Final List:")

#checker_chain.run("You are a master graph modeler, you have to remove edges from an edge list that contain inaccesible nodes. We have a directed graph. It contains exclusively these edges: "+ str([(triple[0], triple[1]) for triple in G.get_triples()]) +"Given that 'N2' is not accesible anymore. Could you provide me with the edges that should be removed, as comma separated values like this [(Node1,Node2),...]?")
#ssc.run("What are the options to get from Charging Station to Waste Disposal? Can you list the routes step by step: Node1->Node2...")
#str([(triple[0], triple[1]) for triple in G.get_triples()])+

#chain.invoke("Assume that node N2 is not available any more. Could you update the list of routes from Charging Station to Waste Disposal : {(\'Edge1\':\'ConnectingEdge1\'),(\'Edge2\':\'ConnectingEdge2\')...\}?")

[('Charging Station', 'N1'), ('N1', 'Mensa'), ('N1', 'N2'), ('Mensa', 'N3'), ('N2', 'N3'), ('N2', 'Blood Storage'), ('N2', 'N4'), ('N3', 'N4'), ('N3', 'Waste Disposal')]
 Node 'N2' is not accesible anymore. Could you provide me only with the edges that have to be removed, as a comma separated value list of tuples like (Node1,Node2)?
Certainly! Since node 'N2' is no longer accessible, we need to remove the edges that connect to it. Here are the edges that need to be removed:

((N1, N2), (Mensa, N2), (N2, Blood Storage), (N2, Waste Disposal))

So, the remaining edges in the list after removing the inaccessible edges are:

([('Charging Station', 'N1'), ('N1', 'Mensa'), ('Mensa', 'N3'), ('N3', 'Waste Disposal')])
List of removed edges: ['((N1, N2), (Mensa, N2), (N2, Blood Storage), (N2, Waste Disposal))', "([('Charging Station', 'N1'), ('N1', 'Mensa'), ('Mensa', 'N3'), ('N3', 'Waste Disposal')])"]
['((N1, N2), (Mensa, N2), (N2, Blood Storage), (N2, Waste Disposal))', "([('Charging Station'

In [326]:
# create chain
from langchain import LLMChain
from langchain.chains import SimpleSequentialChain

# create chain
chain = ({"question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
)

prompt
new_graph=LLMChain(prompt=ChatPromptTemplate.from_template("Assume that node N2 is not available any more. Could you update the list of edges, use this format: {(\'Edge1\':\'ConnectingEdge1\'),(\'Edge2\':\'ConnectingEdge2\')...\}?"),llm=model)
new_route=LLMChain(prompt=ChatPromptTemplate.from_template("{edges},Could you update the list of routes from Charging Station to Waste Disposal based on the new edges given"),llm=model)
sschain = SimpleSequentialChain(chains=[new_graph,new_route],verbose=True)
print(sschain.invoke(''))



> Entering new SimpleSequentialChain chain...


ValueError: Invalid format specifier ''ConnectingEdge1'),('Edge2':'ConnectingEdge2')...\' for object of type 'str'

In [ ]:

route_template="{edges},What are the options to get from Charging Station to Waste Disposal? Can you list the routes step by step: Node1->Node2...?"
prompt_route=PromptTemplate(input_variables=["edges"],template=route_template)
no_input_route=LLMChain(prompt=prompt_route,llm=model)

template_n2="{edges}, Assume that node N2 is not available any more. Update the given list of edges given "
prompt_template = PromptTemplate(input_variables=["edges"], template=template_n2)
remove_n2=LLMChain(prompt=prompt_template,llm=model)

In [ ]:
import pandas as pd

df = pd.read_csv('../Playground_Arved/edges_UH_Graph.csv')

# Transform DataFrame into list of tuples
edges_list = [(f'{row[0]}', f'{row[1]}') for _, row in df.iterrows()]

print(edges_list)

print([tuple for tuple in edges_list])

[('31404364', '7120224687'), ('31404364', '313157654'), ('31404364', '7112240026'), ('31404364', '7120224692'), ('7120224687', '7112240050'), ('7112240050', '7112240049'), ('7112240049', '7123481411'), ('7123481411', '313157656'), ('313157656', '313157659'), ('313157659', '31404365'), ('31404365', '1280377626'), ('1280377626', '313157654'), ('313157654', '7112240047'), ('313157654', '1213060725'), ('7112240047', '7112240051'), ('7112240051', '1280377624'), ('1280377624', '31404366'), ('31404366', '31404367'), ('31404367', '31404363'), ('31404367', '7120224688'), ('31451750', '313316730'), ('31451750', '7273047832'), ('31451750', '245411481'), ('313316730', '245411483'), ('245411483', '245004114'), ('245411483', '245411480'), ('245411483', '245411472'), ('245004114', '245004115'), ('245004114', '245411482'), ('245004114', '245411472'), ('245004115', '313158259'), ('245004120', '7112240058'), ('7112240058', '7037192308'), ('7037192308', '7037192307'), ('7037192308', '7112240052'), ('7037

C:\Users\User\AppData\Local\Temp\ipykernel_6668\1776594052.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  edges_list = [(f'{row[0]}', f'{row[1]}') for _, row in df.iterrows()]
